In [1]:
# Here we want to set up the full set of tasks that we will want to perform
import numpy as np

from matplotlib import pyplot as plt

from findiff import FinDiff

In [2]:
# Set up the parameter space
order = 3

Nparams = 3 # number of parameters
x0s = [1,2,3] # the center and grid spacing
dxs = [0.01, 0.05, 0.01]

output_shape = (2,) # this is the shape of the output e.g. for pkells (nells, nks)

In [3]:
# Now we want to set up the actual grid itself
template = np.arange(-order,order+1,1)
Npoints = 2*order + 1
grid_axes = [ dx*template + x0 for x0, dx in zip(x0s,dxs)]

In [4]:
Inds   = np.meshgrid( * (np.arange(Npoints),)*Nparams, indexing='ij')
Inds = [ind.flatten() for ind in Inds]
center_ii = (order,)*Nparams
Coords = np.meshgrid( *grid_axes, indexing='ij')

In [5]:
# Define the function we want to emulate.... just a trial one for now
def F(pars):
    
    x, y, z = pars
    
    F1 = 1 + (x-1) + (y-2)**2 + 2*(z-3)**3
    F2 = (y-2)*(z-3)
    
    return [F1,F2]

In [6]:
Fs = np.zeros( (Npoints,)*Nparams + output_shape )
for nn, iis in enumerate(zip(*Inds)):
    coord = [Coords[i][iis] for i in range(Nparams)]
    Fs[iis] = F(coord)

In [7]:
# Check if we did this right:
f1 = 1 + (Coords[0]-1) + (Coords[1]-2)**2 + 2*(Coords[2]-3)**3
np.max(np.abs(Fs[:,:,:,0] - f1))

0.0

In [8]:
Fs.shape[len(dxs):]

(2,)

In [9]:
# Now define the derivatives
derivs = []

for oo in range(order+1):
    if oo == 0:
        derivs += [Fs[center_ii]]
    else:
        dnFs = np.zeros( (Nparams,)*oo + output_shape)
        
        # Want to get a list of all the possible d/dx_i dx_j dx_k ...
        param_inds = np.meshgrid( * (np.arange(Nparams),)*oo, indexing='ij')
        PInds = [ind.flatten() for ind in param_inds] 
        
        for iis in zip(*PInds):
            # build a string of (xk, dxk, 1) for taking the d/dxk derivative in sequence
            deriv_tuple = []
            for ii in iis:
                deriv_tuple += [(ii, dxs[ii],1),]
            
            dndx = FinDiff(*deriv_tuple)
            
            dnFs[iis] += dndx(Fs)[center_ii]
        
        derivs += [dnFs]

In [10]:
# Test our function
from taylor_approximation import compute_derivatives

test = compute_derivatives(Fs, dxs, center_ii, order)

In [11]:
# Now let's see if we can recover the function from the stored derivatives
from scipy.special import factorial
test_point = [2,5,4]

Fapprox = np.zeros(output_shape)

for oo in range(order+1):
    if oo == 0:
        Fapprox += derivs[oo]
    else:
        param_inds = np.meshgrid( * (np.arange(Nparams),)*oo, indexing='ij')
        PInds = [ind.flatten() for ind in param_inds] 
        
        for iis in zip(*PInds):
            monomials = [ (test_point[ii] - x0s[ii]) for ii in iis ]
            Fapprox += 1/factorial(oo) * derivs[oo][iis] * np.prod(monomials)
        
    

In [12]:
Fapprox

array([13.0002,  3.    ])

In [13]:
F(test_point)

[13, 3]

In [14]:
from taylor_approximation import taylor_approximate

In [15]:
taylor_approximate(test_point, x0s, derivs)

Taking the order 3 Taylor series.


array([13.0002,  3.    ])

In [19]:
import time
t1 = time.time()
taylor_approximate(test_point, x0s, derivs,order=3)
t2 = time.time()
print(t2-t1)

0.0017445087432861328


In [18]:
t1 = time.time()
F(test_point)
t2 = time.time()
print(t2-t1)

7.319450378417969e-05
